In [1]:
import torch as t
import torchvision
from torchvision import datasets,transforms
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32,padding=1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])
transforms_test = transforms.Compose([
    transforms.RandomCrop(32,padding=1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

In [3]:
train_data = datasets.CIFAR10(root='./cs231n/datasets',train=True,download=False,transform=transform_train)
test_data = datasets.CIFAR10(root='./cs231n/datasets',train=False,download=False,transform=transforms_test)

In [4]:
Batch_size = 64
train_loader = Data.DataLoader(train_data,batch_size=Batch_size,shuffle=True)
test_loader = Data.DataLoader(test_data,batch_size=Batch_size,shuffle=False)

In [5]:
t.cuda.set_device(0)

In [6]:
cfg = [64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']

In [8]:
def make_layers(cfg):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(2)]
        else:
            layers +=[nn.Conv2d(in_channels,v,3,padding=1),nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

In [9]:
class vgg16(nn.Module):
    def __init__(self,feature,data_num,num_classes):
        super(vgg16,self).__init__()
        self.feature = feature
        self.classifier = nn.Sequential(
            nn.Linear(512,256),
            nn.Dropout(0.5),
            nn.Linear(256,128),
            nn.Dropout(0.5),
            nn.Linear(128,num_classes),
        )
    def forward(self,x):
        x = self.feature(x)
        x = x.view(x.size()[0],-1)
        x = self.classifier(x)
        return x

In [10]:
print(train_data)


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./cs231n/datasets
    Split: Train


In [11]:
feature = make_layers(cfg)
data_num = 64
model = vgg16(feature,data_num,10).cuda()


In [12]:
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9, weight_decay=5e-4)


In [13]:
import torch.nn.functional as F

In [14]:
def train_process(model,optimizer,epoches = 1):
    model.train()
    for e in range(epoches):
        for i,data in enumerate(train_loader):
            
            x, y = data
            x, y = Variable(x).cuda(), Variable(y).cuda()
            
            optimizer.zero_grad()
            scores = model(x)
#             print(scores.shape,y.shape)
            loss = F.cross_entropy(scores,y)
            
            
            loss.backward()
            optimizer.step()
            
            if(i%10==0):
                print("iteration [%d/%d] loss:%.03f"%(e,epoches,loss.item()))

In [16]:
train_process(model,optimizer=optimizer,epoches=10)

iteration [0/10] loss:0.290
iteration [0/10] loss:0.323
iteration [0/10] loss:0.468
iteration [0/10] loss:0.368
iteration [0/10] loss:0.578
iteration [0/10] loss:0.502
iteration [0/10] loss:0.448
iteration [0/10] loss:0.345
iteration [0/10] loss:0.496
iteration [0/10] loss:0.239
iteration [0/10] loss:0.386
iteration [0/10] loss:0.385
iteration [0/10] loss:0.384
iteration [0/10] loss:0.206
iteration [0/10] loss:0.301
iteration [0/10] loss:0.273
iteration [0/10] loss:0.281
iteration [0/10] loss:0.228
iteration [0/10] loss:0.198
iteration [0/10] loss:0.146
iteration [0/10] loss:0.295
iteration [0/10] loss:0.321
iteration [0/10] loss:0.239
iteration [0/10] loss:0.304
iteration [0/10] loss:0.375
iteration [0/10] loss:0.152
iteration [0/10] loss:0.423
iteration [0/10] loss:0.183
iteration [0/10] loss:0.595
iteration [0/10] loss:0.464
iteration [0/10] loss:0.218
iteration [0/10] loss:0.358
iteration [0/10] loss:0.260
iteration [0/10] loss:0.386
iteration [0/10] loss:0.343
iteration [0/10] los

iteration [3/10] loss:0.230
iteration [3/10] loss:0.362
iteration [3/10] loss:0.356
iteration [3/10] loss:0.363
iteration [3/10] loss:0.249
iteration [3/10] loss:0.218
iteration [3/10] loss:0.333
iteration [3/10] loss:0.338
iteration [3/10] loss:0.330
iteration [3/10] loss:0.351
iteration [3/10] loss:0.172
iteration [3/10] loss:0.311
iteration [3/10] loss:0.241
iteration [3/10] loss:0.167
iteration [3/10] loss:0.152
iteration [3/10] loss:0.209
iteration [3/10] loss:0.250
iteration [3/10] loss:0.303
iteration [3/10] loss:0.507
iteration [3/10] loss:0.347
iteration [3/10] loss:0.454
iteration [3/10] loss:0.311
iteration [3/10] loss:0.173
iteration [4/10] loss:0.195
iteration [4/10] loss:0.262
iteration [4/10] loss:0.303
iteration [4/10] loss:0.205
iteration [4/10] loss:0.212
iteration [4/10] loss:0.304
iteration [4/10] loss:0.187
iteration [4/10] loss:0.282
iteration [4/10] loss:0.329
iteration [4/10] loss:0.254
iteration [4/10] loss:0.407
iteration [4/10] loss:0.286
iteration [4/10] los

iteration [7/10] loss:0.249
iteration [7/10] loss:0.198
iteration [7/10] loss:0.191
iteration [7/10] loss:0.090
iteration [7/10] loss:0.164
iteration [7/10] loss:0.118
iteration [7/10] loss:0.215
iteration [7/10] loss:0.445
iteration [7/10] loss:0.285
iteration [7/10] loss:0.225
iteration [7/10] loss:0.120
iteration [7/10] loss:0.371
iteration [7/10] loss:0.222
iteration [7/10] loss:0.175
iteration [7/10] loss:0.273
iteration [7/10] loss:0.135
iteration [7/10] loss:0.232
iteration [7/10] loss:0.147
iteration [7/10] loss:0.165
iteration [7/10] loss:0.278
iteration [7/10] loss:0.261
iteration [7/10] loss:0.287
iteration [7/10] loss:0.314
iteration [7/10] loss:0.294
iteration [7/10] loss:0.100
iteration [7/10] loss:0.306
iteration [7/10] loss:0.125
iteration [7/10] loss:0.211
iteration [7/10] loss:0.159
iteration [7/10] loss:0.171
iteration [7/10] loss:0.255
iteration [7/10] loss:0.219
iteration [7/10] loss:0.188
iteration [7/10] loss:0.158
iteration [7/10] loss:0.230
iteration [7/10] los

In [17]:
def test_process(model):
    model.eval()
    loss, num = 0, 0
    for i, data in enumerate(test_loader):
        x, y = data
        x, y = Variable(x).cuda(), Variable(y).cuda()
        
        scores = model(x)
        _,pre = scores.max(1)
        
        loss += (pre==y).sum()
        num += scores.size()[0]
        
        acc = float(loss)/num
        print("[%d / %d] correct (%.2f)"%(loss,num,acc))

In [18]:
test_process(model)

[58 / 64] correct (0.91)
[114 / 128] correct (0.89)
[169 / 192] correct (0.88)
[223 / 256] correct (0.87)
[277 / 320] correct (0.87)
[330 / 384] correct (0.86)
[385 / 448] correct (0.86)
[444 / 512] correct (0.87)
[501 / 576] correct (0.87)
[562 / 640] correct (0.88)
[615 / 704] correct (0.87)
[670 / 768] correct (0.87)
[720 / 832] correct (0.87)
[776 / 896] correct (0.87)
[827 / 960] correct (0.86)
[885 / 1024] correct (0.86)
[942 / 1088] correct (0.87)
[995 / 1152] correct (0.86)
[1051 / 1216] correct (0.86)
[1107 / 1280] correct (0.86)
[1164 / 1344] correct (0.87)
[1224 / 1408] correct (0.87)
[1281 / 1472] correct (0.87)
[1336 / 1536] correct (0.87)
[1392 / 1600] correct (0.87)
[1446 / 1664] correct (0.87)
[1499 / 1728] correct (0.87)
[1550 / 1792] correct (0.86)
[1605 / 1856] correct (0.86)
[1659 / 1920] correct (0.86)
[1710 / 1984] correct (0.86)
[1762 / 2048] correct (0.86)
[1814 / 2112] correct (0.86)
[1868 / 2176] correct (0.86)
[1919 / 2240] correct (0.86)
[1972 / 2304] correc